![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import re #
from sklearn.preprocessing import MultiLabelBinarizer # Для создания Dummy из списков

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

Создадим несколько функций для дальнейшей обработки

In [ ]:

def get_cuisines_list(s):
    '''Используем для преобразования строки в список'''
    if bool(s):
        result = s.replace('[', '').replace(']', '').replace(
            '"', '').replace("'", '').split(', ')
    else:
        result = []
    return result

def get_dates(s, c):
    '''Используем для извлечения дат обзоров из "Reviews"'''
    result = ''
    if s!='[[], []]':
        if len(s) == 0:
            result = ''
        else:
            s = re.findall(r'\d{2}/\d{2}/\d{4}', s)
            if c == 1:
                result = s[0]
            elif len(s) == 2:
                result = s[1]
            else:
                result = ''
    return result

def get_revs(s, c):
    '''Используем для извлечения текста обзоров из "Reviews"'''
    s = re.match(r"\[\[(.*)\],", s).group()[2:-2]
    if c == 1:
        result = s
        if bool(re.search(r"\'(.*)\', ", s)):
            result = re.search(r"\'(.*)\', ", s).group()[:-2]
    else:
        result = ''
        if bool(re.search(r", [\'\"](.*)[\'\"]", s)):
            result = re.search(r", [\'\"](.*)[\'\"]", s).group()[2:]
    return result

def order_revs(row):
    '''Используем для упорядочения хронологии обзоров'''
    if bool(row.DateRev2):
        if row.DateRev2 > row.DateRev1:
            row.Rev2, row.Rev1, row.DateRev2, row.DateRev1 = row.Rev1, row.Rev2, row.DateRev1, row.DateRev2
    return row

def get_tone(s):
    '''Используем для определения тональности отзыва'''
    if pd.notna(s):
        result = sum(map(lambda word: afinn.get(word, 0),
                         re.sub(r'[^\w ]', '', s).lower().split()))
    else:
        result = s
    return result

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Обработка NaN
data['Number_of_Reviews_isNAN'] = pd.isna(
    data['Number of Reviews']).astype('uint8')
data["Reviews_isNAN"] = pd.isna(data["Reviews"]).astype("uint8")
data["Reviews"].fillna("[[], []]", inplace=True)
data["Price_Range_isNAN"] = pd.isna(data["Price Range"]).astype("uint8")
data["Cuisine Style_isNAN"] = pd.isna(data["Cuisine Style"]).astype("uint8")

In [ ]:
# Заполним пропуски Cuisine Style наиболее вероятными значениями для данного города.
tdata = data.copy()  # используем временный датафрейм для последующего "взрывания"
tdata.dropna(subset=["Cuisine Style"], inplace=True)
# Преобразуем строковые значения в список для "взрывания"
tdata['Cuisine_list'] = tdata['Cuisine Style'].apply(get_cuisines_list)
tdata = tdata.explode("Cuisine_list")
# Получим список самых популярных кухонь для каждого города
Cousine_by_city = pd.DataFrame(tdata.groupby("City")["Cuisine_list"].agg(
    pd.Series.mode))
Cousine_by_city.columns = ['Pop_city_cuisine']
Cousine_by_city=Cousine_by_city.reset_index()
# Добавим информацию о городах и кухнях в наш датафрейм
data = data.merge(Cousine_by_city, on='City', how='left')
# Заполним пропуски самым популярным значением кухни в данном городе
data['Cuisine Style'] = data['Cuisine Style'].fillna(data['Pop_city_cuisine'])
# Получим данные по кухням в виде столбца списков
data['Cuisine_list'] = data['Cuisine Style'].apply(get_cuisines_list)
# Образуем новый числовой признак - количество видов кухни в ресторане
data['Cuisines_Number'] = data['Cuisine_list'].apply(lambda x: len(x))

In [ ]:
# Извлечем даты обзоров из столбца 'Reviews'
data['DateRev1'] = data['Reviews'].apply(lambda x: get_dates(x, 1))
data['DateRev2'] = data['Reviews'].apply(lambda x: get_dates(x, 2))
data['DateRev1'] = pd.to_datetime(data['DateRev1'].dropna())
data['DateRev2'] = pd.to_datetime(data['DateRev2'].dropna())

# Извлечем текст обзоров
data['Rev1'] = data['Reviews'].apply(lambda x: get_revs(x, 1))
data['Rev2'] = data['Reviews'].apply(lambda x: get_revs(x, 2))

# Расположим обзоры в хронологическом порядке
data[['Rev1', 'Rev2', 'DateRev1', 'DateRev2']] = data[[
    'Rev1', 'Rev2', 'DateRev1', 'DateRev2']].apply(order_revs, 1)

# Получим новый признак - количество дней между обзорами
data['DateDelta'] = (data['DateRev1']-data['DateRev2']).dt.days

# Обработаем пропуски с сохранением информации о них
data['DateDelta_isNAN'] = pd.isna(data['DateDelta']).astype("uint8")
data['DateDelta'].fillna(data['DateDelta'].dropna().median(), inplace=True)

# Определим актуальность отзывов относительно самого свежего встречаемого в базе.
last_date = data[['DateRev1', 'DateRev2']].max().max()
data['Isuptodate'] = (last_date-data['DateRev1']).dt.days
data['Isuptodate'].fillna(data['Isuptodate'].dropna().median(), inplace=True)

# Добавим информацию о количестве видов кухни в разных городах в наш датафрейм
Cuisines_num_by_city = data.explode("Cuisine_list").groupby(
    "City")["Cuisine_list"].unique().apply(len).to_frame().reset_index()
Cuisines_num_by_city.columns = ['City', 'Cuisines_num_by_city']
data = data.merge(Cuisines_num_by_city, on='City', how='left')

# Добавим информацию о медиане отзывов по городам в наш датафрейм для заполнения пропусков в 'Number of Reviews'
Reviews_num_by_city = data.dropna(subset=['Number of Reviews']).groupby(
    "City")['Number of Reviews'].median().to_frame().reset_index()
Reviews_num_by_city.columns = ['City', 'Reviews_num_by_city']
data = data.merge(Reviews_num_by_city, on='City', how='left')
# Заполним пропуски 'Number of Reviews' значениями медианы отзывов по городам
data['Number of Reviews'].fillna(data['Reviews_num_by_city'], inplace=True)

In [ ]:
data.head(5)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
#data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
cities_encoded = pd.get_dummies(data, columns=['City'])
data = pd.concat([data['City'], cities_encoded], axis=1)

In [ ]:
data.info()

In [ ]:
# Закодируем признак 'Cuisine Style' в dummy значения

mlb = MultiLabelBinarizer()
cuisines_encoded = pd.DataFrame(mlb.fit_transform(
    data['Cuisine_list']), columns=mlb.classes_).astype("uint8")
data = pd.concat([data, cuisines_encoded], axis=1)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'
data["Price Range"].fillna(data["Price Range"].mode()[0], inplace=True)
data['Price Range'] = data['Price Range'].replace(
    ['$', '$$ - $$$', '$$$$'], [1, 2, 3])

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
# тут ваш код на обработку других признаков
# Получим оценку тональности отзывов в качестве нового признака
# Загрузим словарь для определения настроения отзывов (http://www2.imm.dtu.dk/pubdb/pubs/6010-full.html)
# Каждое слово словаря имеет оценку (-5:+5). Фраза оценивается как сумма оценок слов. 
afinn = {}
with open("../input/mydict/AFINN-111.txt") as file:
    for line in file:
        key, *value = line.split('\t')
        afinn[key] = int(value[0].replace('\n', ''))
# Рассчитаем тональности отзывов.        
data['Rev1_tone'] = data['Rev1'].apply(get_tone)
data['Rev2_tone'] = data['Rev2'].apply(get_tone)  
# Обработаем пропуски
data['Rev2_tone'].fillna(data['Rev1_tone'], inplace=True)
Rev_tones_mode = data['Rev1_tone'].mode()[0]
data['Rev1_tone'].fillna(Rev_tones_mode, inplace=True)
data['Rev2_tone'].fillna(Rev_tones_mode, inplace=True)


![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

In [ ]:
# Нормализуем 'Ranking' для каждого города и добавим новый признак в датафрейм
data['Rank_Norm'] = data.groupby('City')['Ranking'].rank(pct=True)
sns.distplot(data["Rank_Norm"], kde=False)
plt.show()

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
# Для построения диаграммы корреляций используем только числовые и ранговые признаки
cols = ['City', 'Ranking', 'Price Range', 'Number of Reviews', 'Rank_Norm',
        'Reviews_num_by_city', 'DateDelta',
        'Rev1_tone', 'Rev2_tone', 'Cuisines_Number', 'Isuptodate',
        'Cuisines_num_by_city', 'Number_of_Reviews_isNAN', 
        'Price_Range_isNAN', 'Cuisine Style_isNAN', 'DateDelta_isNAN', 'Rating']
corr = data[data['sample']==1][cols].corr(method="spearman")

sns.heatmap(corr, annot=True, fmt=".2f")
plt.show()

In [ ]:
np.abs(corr["Rating"].drop("Rating")).sort_values(ascending=False)

In [ ]:
data.info(verbose=True)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA','URL_TA'], axis = 1, inplace=True)
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    # Обработка NaN
    df_output['Number_of_Reviews_isNAN'] = pd.isna(
    df_output['Number of Reviews']).astype('uint8')
    df_output['Reviews_isNAN'] = pd.isna(df_output["Reviews"]).astype("uint8")
    df_output["Reviews"].fillna("[[], []]", inplace=True)
    df_output["Price_Range_isNAN"] = pd.isna(df_output["Price Range"]).astype("uint8")
    df_output["Cuisine Style_isNAN"] = pd.isna(df_output["Cuisine Style"]).astype("uint8")
    
    # Заполним пропуски Cuisine Style наиболее вероятными значениями для данного города.
    tdf_output = df_output.copy()  # используем временный датафрейм для последующего "взрывания"
    tdf_output.dropna(subset=["Cuisine Style"], inplace=True)
    # Преобразуем строковые значения в список для "взрывания"
    tdf_output['Cuisine_list'] = tdf_output['Cuisine Style'].apply(get_cuisines_list)
    tdf_output = tdf_output.explode("Cuisine_list")
    # Получим список самых популярных кухонь для каждого города
    Cousine_by_city = pd.DataFrame(tdf_output.groupby("City")["Cuisine_list"].agg(
        pd.Series.mode))
    Cousine_by_city.columns = ['Pop_city_cuisine']
    Cousine_by_city=Cousine_by_city.reset_index()
    # Добавим информацию о городах и кухнях в наш датафрейм
    df_output = df_output.merge(Cousine_by_city, on='City', how='left')
    # Заполним пропуски самым популярным значением кухни в данном городе
    df_output['Cuisine Style'] = df_output['Cuisine Style'].fillna(df_output['Pop_city_cuisine'])
    # Получим данные по кухням в виде столбца списков
    df_output['Cuisine_list'] = df_output['Cuisine Style'].apply(get_cuisines_list)
    # Образуем новый числовой признак - количество видов кухни в ресторане
    df_output['Cuisines_Number'] = df_output['Cuisine_list'].apply(lambda x: len(x))
    
    # Извлечем даты обзоров из столбца 'Reviews'
    df_output['DateRev1'] = df_output['Reviews'].apply(lambda x: get_dates(x, 1))
    df_output['DateRev2'] = df_output['Reviews'].apply(lambda x: get_dates(x, 2))
    df_output['DateRev1'] = pd.to_datetime(df_output['DateRev1'].dropna())
    df_output['DateRev2'] = pd.to_datetime(df_output['DateRev2'].dropna())

    # Извлечем текст обзоров
    df_output['Rev1'] = df_output['Reviews'].apply(lambda x: get_revs(x, 1))
    df_output['Rev2'] = df_output['Reviews'].apply(lambda x: get_revs(x, 2))

    # Расположим обзоры в хронологическом порядке
    df_output[['Rev1', 'Rev2', 'DateRev1', 'DateRev2']] = df_output[[
        'Rev1', 'Rev2', 'DateRev1', 'DateRev2']].apply(order_revs, 1)

    # Получим новый признак - количество дней между обзорами
    df_output['DateDelta'] = (df_output['DateRev1']-df_output['DateRev2']).dt.days

    # Обработаем пропуски с сохранением информации о них
    df_output['DateDelta_isNAN'] = pd.isna(df_output['DateDelta']).astype("uint8")
    df_output['DateDelta'].fillna(df_output['DateDelta'].dropna().median(), inplace=True)

    # Определим актуальность отзывов относительно самого свежего встречаемого в базе.
    last_date = df_output[['DateRev1', 'DateRev2']].max().max()
    df_output['Isuptodate'] = (last_date-df_output['DateRev1']).dt.days
    df_output['Isuptodate'].fillna(df_output['Isuptodate'].dropna().median(), inplace=True)

    # Добавим информацию о количестве видов кухни в разных городах в наш датафрейм
    Cuisines_num_by_city = df_output.explode("Cuisine_list").groupby(
        "City")["Cuisine_list"].unique().apply(len).to_frame().reset_index()
    Cuisines_num_by_city.columns = ['City', 'Cuisines_num_by_city']
    df_output = df_output.merge(Cuisines_num_by_city, on='City', how='left')

    # Добавим информацию о медиане отзывов по городам в наш датафрейм для заполнения пропусков в 'Number of Reviews'
    Reviews_num_by_city = df_output.dropna(subset=['Number of Reviews']).groupby(
        "City")['Number of Reviews'].median().to_frame().reset_index()
    Reviews_num_by_city.columns = ['City', 'Reviews_num_by_city']
    df_output = df_output.merge(Reviews_num_by_city, on='City', how='left')
    # Заполним пропуски 'Number of Reviews' значениями медианы отзывов по городам
    df_output['Number of Reviews'].fillna(df_output['Reviews_num_by_city'], inplace=True)
    
    # ################### 3. Encoding ############################################################## 
    # Закодируем признак 'City' в dummy значения
    cities_encoded = pd.get_dummies(df_output, columns=['City'])
    df_output = pd.concat([df_output['City'], cities_encoded], axis=1)
    
    # Закодируем признак 'Cuisine Style' в dummy значения
    mlb = MultiLabelBinarizer()
    cuisines_encoded = pd.DataFrame(mlb.fit_transform(
        df_output['Cuisine_list']), columns=mlb.classes_).astype("uint8")
    df_output = pd.concat([df_output, cuisines_encoded], axis=1)
    

    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    
    # Нормализуем 'Ranking' для каждого города и добавим новый признак в датафрейм
    df_output['Rank_Norm'] = df_output.groupby('City')['Ranking'].rank(pct=True)
    
    # Обработка 'Price Range' в ординальный признак
    df_output["Price Range"].fillna(df_output["Price Range"].mode()[0], inplace=True)
    df_output['Price Range'] = df_output['Price Range'].replace(
        ['$', '$$ - $$$', '$$$$'], [1, 2, 3])
    
    # Получим оценку тональности отзывов в качестве нового признака
    # Загрузим словарь для определения настроения отзывов (http://www2.imm.dtu.dk/pubdb/pubs/6010-full.html)
    # Каждое слово словаря имеет оценку (-5:+5). Фраза оценивается как сумма оценок слов. 
    afinn = {}
    with open("../input/mydict/AFINN-111.txt") as file:
        for line in file:
            key, *value = line.split('\t')
            afinn[key] = int(value[0].replace('\n', ''))
    # Рассчитаем тональности отзывов.        
    df_output['Rev1_tone'] = df_output['Rev1'].apply(get_tone)
    df_output['Rev2_tone'] = df_output['Rev2'].apply(get_tone)  
    # Обработаем пропуски
    df_output['Rev2_tone'].fillna(df_output['Rev1_tone'], inplace=True)
    Rev_tones_mode = df_output['Rev1_tone'].mode()[0]
    df_output['Rev1_tone'].fillna(Rev_tones_mode, inplace=True)
    df_output['Rev2_tone'].fillna(Rev_tones_mode, inplace=True)
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    df_output.drop(['DateRev1','DateRev2'], axis = 1, inplace=True)
    
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info(verbose=True)

In [ ]:
df_preproc[['Ranking','Price Range', 'Number of Reviews', 'Cuisines_Number', 'DateDelta', 'Isuptodate', 'Cuisines_num_by_city', 'Reviews_num_by_city' ]]=df_preproc[['Ranking','Price Range', 'Number of Reviews', 'Cuisines_Number', 'DateDelta', 'Isuptodate', 'Cuisines_num_by_city', 'Reviews_num_by_city' ]].astype('int64')

In [ ]:
df_preproc.info(verbose=True)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
